## Desenvolvimento de indicador IFR (Índice de Força Relativa) - Modelo RSI Cutler

In [1]:
# Instalar MetaTrader5

!pip install MetaTrader5

In [2]:
# Análise e manipulação de dados

import numpy as np
import pandas as pd

In [3]:
# Tratamento de séries temporais

from datetime import datetime
import pytz
import time

In [4]:
# Integração com o MetaTrader5

import MetaTrader5 as mt5

In [5]:
import seaborn as sns

In [8]:
# Inicializar contato MetraTrader5

mt5.initialize()

In [9]:
# Selecionar o Ativo
symbol3 = "WIN$"
# Selecionar timezone para São Paulo
timezone = pytz.timezone("America/sao_Paulo")
# Definir data inicial e data final
data_inicio = datetime(2022,1,1, tzinfo=timezone)
data_fim = datetime.today()
# Criar objeto "cotacoes" com os dados do período
cotacoes = mt5.copy_rates_range(symbol3, mt5.TIMEFRAME_M1, data_inicio, data_fim)
# Criar dataframe a partir dos dados do objeto cotacoes
df_IFR = pd.DataFrame(cotacoes)
# Transformar a coluna "time" para formato de data e hora padrão
df_IFR['time'] = pd.to_datetime(df_IFR['time'], unit = 's')
df_IFR.index = df_IFR['time']
# Criar nova coluna "difenca_close". 
# Calcular a diferença entre as colunas de fechamento 'close' em nova coluna criada.
df_IFR['diferenca_close'] = df_IFR['close'].diff()
# Filtrar valores > 0 em coluna 'diferenca_close' e colocá-los em nova coluna 'difenca_pos'
# Em nova coluna 'difenca_pos', trocar 'Nan' por '0'
df_IFR.loc[df_IFR['diferenca_close'] > 0, 'diferenca_pos'] = df_IFR['diferenca_close']
df_IFR['diferenca_pos'] = df_IFR['diferenca_pos'].fillna(0)
# Filtrar valores < 0 em coluna 'diferenca_close' e colocá-los em nova coluna 'difenca_neg'
# Em nova coluna 'difenca_neg', trocar 'Nan' por '0'
df_IFR.loc[df_IFR['diferenca_close'] < 0, 'diferenca_neg'] = abs(df_IFR['diferenca_close'])
df_IFR['diferenca_neg'] = df_IFR['diferenca_neg'].fillna(0)
# Criar nova coluna 'media_pos' e calcular valor de média, janela 14 períodos, de valores coluna 'diferenca_pos'
# Em nova coluna 'media_pos', trocar 'Nan' por '0'
df_IFR['media_pos'] = df_IFR['diferenca_pos'].rolling(window = 14).mean()
df_IFR['media_pos'] = df_IFR['media_pos'].fillna(0)
# Criar nova coluna 'media_neg' e calcular valor de média, janela 14 períodos, de valores coluna 'diferenca_neg'
# Em nova coluna 'media_neg', trocar 'Nan' por '0'
df_IFR['media_neg'] = df_IFR['diferenca_neg'].rolling(window = 14).mean()
df_IFR['media_neg'] = df_IFR['media_neg'].fillna(0)
# Criar coluna IFR e realizar cálculo valor IFR Cutler
df_IFR['IFR'] = (100-(100/(1+(df_IFR['media_pos']/df_IFR['media_neg']))))
df_IFR.tail()

,time,open,high,low,close,tick_volume,spread,real_volume,diferenca_close,diferenca_pos,diferenca_neg,media_pos,media_neg,IFR
time,,,,,,,,,,,,,,
2023-07-18 15:36:00,2023-07-18 15:36:00,119150.0,119165.0,119110.0,119130.0,1198,0,15013,-20.0,0.0,20.0,13.214286,12.500000,51.388889
2023-07-18 15:37:00,2023-07-18 15:37:00,119125.0,119160.0,119125.0,119145.0,915,0,8900,15.0,15.0,0.0,14.285714,11.785714,54.794521
2023-07-18 15:38:00,2023-07-18 15:38:00,119145.0,119150.0,119130.0,119150.0,602,0,5956,5.0,5.0,0.0,14.642857,11.785714,55.405405
2023-07-18 15:39:00,2023-07-18 15:39:00,119145.0,119220.0,119135.0,119200.0,2126,0,35062,50.0,50.0,0.0,17.142857,11.785714,59.259259
2023-07-18 15:40:00,2023-07-18 15:40:00,119205.0,119280.0,119200.0,119225.0,2010,0,35575,25.0,25.0,0.0,18.928571,9.285714,67.088608


In [10]:
df_IFR['IFR'][-1]

67.0886075949367